# Modeling , Param Tuning, Evulating, Explaining & Prediction

In [348]:
#Loading libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from datetime import datetime
import pickle 
import warnings
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score,log_loss,roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
import category_encoders as ce
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer
import shap
import json
import imblearn

import DATA_PROCESSING as process
%pylab inline
pd.set_option('display.max_rows', 500)

Populating the interactive namespace from numpy and matplotlib


pylab import has clobbered these variables: ['identity', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy


In [349]:
sklearn.__version__

'0.21.3'

# Running parameters

In [350]:
SAMPLE = 1000000
USE_SMOTE = True
CALC_PARAMS = True
dev_mode = True
config_file = "configuration_01_15_2022__08_18_sample_1000000.csv"


In [351]:
columns_cum =['app_cat','manufacturer','device_model','device_version','user_isp']
columns_dummies=['banner_pos', 'Day_of_Week','state','manufacturer','Month','hour','user_isp','app_cat','device_model','device_version']
features = ['state', 'user_isp', 'app_cat', 'banner_pos', 'manufacturer', 'device_model', 'device_version', 'device_height', 'device_width', 'device_diag', 'Day_of_Week', 'Month', 'hour']
cat_features = ['state', 'user_isp', 'app_cat', 'banner_pos', 'manufacturer', 'device_model', 'device_version', 'Day_of_Week', 'Month', 'hour']

# Read Training Data

In [352]:
df = pd.read_pickle("train_set.pickle").sample(n=SAMPLE)
df.loc[:,"clicked"] = df.loc[:,"clicked"].map({True:1, False:0})

### Imputing, Feature engineering,Cardinality reductions , and dummy coding 

In [353]:
def preprocessing_pipeline(df, one_hot_encoding=True, allowed_levels_dict = None):
    df= df.drop(['op_id', 'resolution','app_id'], axis=1)
    # if  a dict of categorical columns and theyr allowed levels is given - replace levels not in the list as "<col_name>_other"
    if allowed_levels_dict:
        for col_name, categories_list in allowed_levels_dict.items():
            df.loc[:, col_name] = df[col_name].map(lambda x: x if x in categories_list else f"{col_name}_other")
    df=process.Missing_values(df)
    df=process.Feature_engineering(df)
    if not allowed_levels_dict:
        df, cat_dict = process.cumulatively_categorise(df,columns_cum)
    if one_hot_encoding:
        df=process.get_dummies_fun(df,columns_dummies)
    return df,allowed_levels_dict if allowed_levels_dict else cat_dict 

In [354]:
%%time
df, allowed_levels_dict = preprocessing_pipeline(df, one_hot_encoding=USE_SMOTE)
MODEL_COLS = list(df.columns)
df.shape

summary of missing values before imputing missing data: 
user_isp            15
manufacturer    306036
device_model    306003
dtype: int64
summary of missing values after imputing missing data based on device version
manufacturer    1283
device_model    1283
dtype: int64
summary of missing values after imputing missing data by assigning a separate category
Series([], dtype: int64)
Wall time: 26 s


(1000000, 245)

# exploring the Target Encoder

for classification tasks Target encoding is a novel and usefull method for converting high-cardinality categorical variable to a continuous value by encoding for each level the propoprion of rows in which Y=1\
i.e. for click prediction it encodes the ctr for each level over the entire dataset



encoder = ce.TargetEncoder(cols=cat_features)
encoded = encoder.fit_transform(df[features], df['clicked'])
encoded.describe()

# look at Class (Im)Balance

In [355]:
df.groupby('clicked').clicked.count()

clicked
0    932181
1     67819
Name: clicked, dtype: int64

# splitting the data test to 3 parts:
1. training: 80%
2. validation: 10% - for param running 
3. test set: 10% a final test set to evaluate the perfromance of the final model, only after all tunning has completed.

In [356]:
train_df, other = train_test_split(df, test_size=0.2, random_state=42)
dev_val, dev_test = train_test_split(other, test_size=0.5, random_state=42)


In [357]:
train_df.shape, dev_val.shape,  dev_test.shape

((800000, 245), (100000, 245), (100000, 245))

# SMOTE oversampling - on the training set only

In [358]:
def appply_smote(df):
    cat_features =  [c for c in df.columns if 'device_' not in c and c != 'clicked']
    features =  [c for c in df.columns if c != 'clicked']
    
    kw_args={
                'y':df['clicked'],
                'X_cols' :list(df[features].columns),
                'y_cols' :list(df[['clicked']].columns)
            }
    smote_transformer = FunctionTransformer(process.Smote_alg,kw_args=kw_args,validate=True)

    smote_pipe = Pipeline([('smote', smote_transformer)])
    new_df, new_y = smote_pipe.fit_transform(df[features])

    #print ('Before', df[features].shape, df['clicked'].shape, 'After', new_df.shape, new_y.shape)
    
    new_df.loc[:, 'clicked'] = new_y['clicked']
    df = new_df[MODEL_COLS]
    cat_features = []
    return df, features, cat_features



In [ ]:
#TODO add the explanation here
if USE_SMOTE:
    train_df, features, cat_features = appply_smote(train_df)
    

org X.shape (800000, 244) len(X_cols) 244
new_X.shape (1491268, 244)


In [ ]:
dev_val.head()

In [ ]:
train_df.info()

In [ ]:
X_train = train_df[features]
y_train = train_df['clicked']
X_val = dev_val[features]
y_val = dev_val['clicked']


In [ ]:
def get_config(model):
    d = configs.loc[model].params
    params = json.loads(d)
    print (params)
    return (params)
configs = pd.read_csv(config_file).set_index('index')
configs

# Preparing for pipeline creation

In [ ]:
def plot_auc(fpr, tpr):
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.3f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()
    return roc_auc

In [ ]:

def plot_calibration(y, y_hat):
    # reliability diagram
    fop, mpv = calibration_curve(y, y_hat, n_bins=10, normalize=True)
    # plot perfectly calibrated
    pyplot.plot([0, 1], [0, 1], linestyle='--')
    # plot model reliability
    pyplot.plot(mpv, fop, marker='.')
    plt.xlabel("Mean predicted probabilities")
    plt.ylabel("True Probabilities")
    plt.title("Calibration Curve")
    pyplot.show()

In [ ]:
def find_threshold_by_accuracy(y, y_hat, plot=False):
    threshold = []
    accuracy = []
    for p in np.sort(np.unique(y_hat))[-50:]:
        threshold.append(p)
        y_pred = (y_hat >= p).astype(int)
        curr_accuracy = accuracy_score(y,y_pred)
        accuracy.append(curr_accuracy)
    best_threshold = threshold[np.argmax(np.array(accuracy), axis=0)]
    best_accuracy = max(accuracy)
    print ("Best Threshold:" ,best_threshold, ", with accuracy = ", best_accuracy)
    if plot:
        plt.scatter(threshold,accuracy)
        plt.xlabel("Threshold")
        plt.ylabel("Accuracy")
        plt.show()
    return best_accuracy, best_threshold

In [ ]:

def find_best_params_random_forest(grid, X_train, y_train, X_val, y_val):
    best_metric = 0
    worst_metric = 1
    for g in ParameterGrid(grid):
        print(g)
        rf_pipe = make_rf_pipe_line(None, params = g, target_encoding = (USE_SMOTE == False))
        rf_pipe.fit(X_train,y_train)
        y_hat = rf_pipe.predict_proba(X_val)[:,1]
        curr_metric, best_threshold = find_threshold_by_accuracy(y_val, y_hat, plot=False)

        # save if best
        if curr_metric > best_metric:
            best_metric = curr_metric
            best_grid = g
            
        if curr_metric < worst_metric:
            worst_metric = curr_metric
            worst_grid = g
    print ("Final Result")
    print ("best metric", best_metric, ", Grid:", best_grid)
    print ("worst metric", worst_metric, ", Grid:", best_grid)
    return best_grid

In [ ]:
def evaulate_pipeline(pipe, X, y, params):
    y_hat = pipe.predict_proba(X)[:,1]
    assert y.shape ==  y_hat.shape
    best_accuracy, best_threshold = find_threshold_by_accuracy(y_val, y_hat, plot=False)
    
    y_pred = (y_hat >= best_threshold).astype(int)

 
    fpr, tpr, thresholds = metrics.roc_curve(y, y_hat, pos_label=1)
    auc = plot_auc(fpr, tpr)
    plot_calibration(y, y_hat)
    #plt.hist(y_hat[y_hat >= 0], bins=10, density=False)
    plt.hist(y_hat, bins=100, density=True)
    plt.title("Histogram of predicted probabilities for click")
    plt.show()
   
    result = {
        "Accuracy": accuracy_score(y, y_pred, normalize=True),
        "Recall": recall_score(y, y_pred),
        "Precision": precision_score(y, y_pred),
        "AUC": auc,
        "-LogLoss(higer is better)": -log_loss(y, y_pred),
        "threshold":   best_threshold,
        "params": json.dumps(params)
    }
    return result
    

In [ ]:
identity  = FunctionTransformer(lambda x:x, validate=True)

# Model 1: Random Forest PipeLine

In [ ]:
def make_rf_pipe_line(calibration = None, params = None , target_encoding = True):
    assert calibration is None or calibration in ['isotonic', 'sigmoid']
    encoder = ce.TargetEncoder(cols=cat_features) if target_encoding else identity
    rf = RandomForestClassifier(n_estimators = 200, max_depth=5, criterion = "gini", n_jobs = 4,
                                min_samples_split=100,random_state=13)
    if params:
        rf.set_params(**params)
    if calibration is None:
        rf_pipe = Pipeline([('target_enc',encoder),  ('classifier', rf)])
    else:
        calibrated_RF_classifier =  CalibratedClassifierCV(base_estimator=rf, cv=5, method=calibration)
        rf_pipe = Pipeline([('target_enc',encoder),  ('calibrated_classifier', calibrated_RF_classifier)])
    rf_pipe.fit(X_train, y_train)
    return rf_pipe


### Parameters being tuned:
1. min_samples_leaf = the minimal number of observations in a new leaf that are required in order for creating it
2. max_depth        = the maximal depth that is allowed in each tree (# splits a in any directed path)
3. class_weight     = the strategy for creating class weights to handle imbalanced data
4. n_estimators     = the totl number of trees in the forest
5. criterion        = the metric for calculating improvement in information by the node splitting: Gini inequality or entropy

In [ ]:
# this is a long step, was executed and saved previously

if CALC_PARAMS:

    if dev_mode:
        rf_params_grid ={'min_samples_leaf': [50, 100], 
                     'max_depth': [3,5] ,
                     'class_weight' : ['balanced'],
                     'n_estimators': [100],
                     'criterion': ["gini"]

                    }     
    else:    
        rf_params_grid ={'min_samples_leaf': [50, 100, 500], 
                         'max_depth': [3,5] ,
                         'class_weight' : ['balanced', None],
                         'n_estimators': [100, 200],
                         'criterion': ["gini"]

                        }
    best_rf_params = find_best_params_random_forest(rf_params_grid, X_train, y_train, X_val, y_val)
else:
    get_config
    best_rf_params = get_config('RF')


In [ ]:

rf_pipe = make_rf_pipe_line(calibration = 'sigmoid' if USE_SMOTE else None,
                            params = best_rf_params,
                            target_encoding = (USE_SMOTE==False))


In [ ]:
rf_results = evaulate_pipeline(rf_pipe, X_val, y_val,best_rf_params)
rf_df = pd.DataFrame(rf_results, index= ['RF'])
rf_df

# Model 2: CatBoost 

In [ ]:
X_val = dev_val[features]
y_val = dev_val['clicked']

In [ ]:
validation_pool = Pool(data = X_val, label = y_val, cat_features=cat_features)
train_pool = Pool(data = X_train, label = y_train, cat_features=cat_features)


In [ ]:
def make_catboost_pipe_line(train_pool, validation_pool , class_weight = None, target_encoder = False, params = None):
    model = CatBoostClassifier(iterations=500,
                               learning_rate=None,
                              cat_features= cat_features ,
                               depth=5,
                               custom_loss = ['AUC', 'Logloss','BrierScore', 'Precision', 'Recall'],
                               early_stopping_rounds = 20,
                               auto_class_weights = class_weight,
                               subsample= 0.5,
                               verbose=30)
    if params:
        model.set_params(**params)
    model.fit(train_pool, eval_set=validation_pool, plot=True)
    return model


### Parameters being tuned:
1. min_child_samples  = the minimal number of observations in a new leaf that are required in order for creating it
2. depth              = the maximal depth that is allowed in each tree (# splits a in any directed path)
3. l2_leaf_reg        = L2 regularization coeffecient  
4. n_estimators       = the totl number of trees in the forest
5. subsample          = proprtion of rows sampled in each tree growin


In [ ]:
if dev_mode:
    catboost_params_grid ={ 
                      'depth': [4,6,8] ,
                      'min_child_samples' : [100,500],
                      'n_estimators': [100],
                      'subsample': [0.95],
                      'l2_leaf_reg':[1],

                    }
else:     
    catboost_params_grid ={ 
                      'depth': [4,6,8] ,
                      'min_child_samples' : [100,500,1000],
                      'n_estimators': [100,200],
                      'subsample': [0.75,0.95],
                      'l2_leaf_reg':[1,2],

                    }


In [ ]:
model = CatBoostClassifier(iterations=500,
                               learning_rate=None,
                              cat_features=cat_features,
                               depth=5,
                               early_stopping_rounds = 20,
                               auto_class_weights = 'Balanced',
                               subsample= 0.5,
                               verbose=30)
if CALC_PARAMS:

    search_results = model.randomized_search(catboost_params_grid,
                      train_pool,
                      y=None,
                      cv=3,
                      n_iter=10,
                      partition_random_seed=0,
                      calc_cv_statistics=True,
                      search_by_train_test_split=True,
                      refit=True,
                      shuffle=True,
                      stratified=None,
                      train_size=0.8,
                      verbose=False,
                      log_cout=sys.stdout,
                  log_cerr=sys.stderr)

    best_catboost_params = search_results['params']
    best_catboost_params

else:
    best_catboost_params = get_config('catboost')

### optimized configuration for catboost

In [ ]:
best_catboost_params

### Train a final cat boost model with the optimized params

In [ ]:
catboost_pipe = make_catboost_pipe_line(train_pool, validation_pool, target_encoder = False, params = best_catboost_params)


In [ ]:
catboost_results = evaulate_pipeline(catboost_pipe, X_val, y_val, best_catboost_params)
catboost_df = pd.DataFrame(catboost_results, index= ['catboost'])
catboost_df

# Model 3: KNN Classifier 

In [ ]:
def make_knn_pipe_line(params = None) :
    encoder = ce.TargetEncoder(cols=cat_features)
    knn_model = KNeighborsClassifier(n_neighbors=300, weights='uniform')
    if params:
        knn_model.set_params(**params)
    return (Pipeline([('target_enc',encoder),  ('scaler',  StandardScaler()), ('KNN', knn_model)]))   

In [ ]:
def find_best_params_knn(grid,X_train, y_train, X_val, y_val):
    best_accuracy = float('-inf')
    worst_accuracy = float('inf')
    for g in ParameterGrid(grid):
        print(g)
        model = make_knn_pipe_line(params = g)
        model.fit(X_train, y_train)
        y_hat = model.predict_proba(X_val)[:,1]
        curr_accuracy, best_threshold = find_threshold_by_accuracy(y_val, y_hat, plot=False)

        # save if best
        if curr_accuracy > best_accuracy:
            best_accuracy = curr_accuracy
            best_grid = g
            
        if curr_accuracy < worst_accuracy:
            worst_accuracy = curr_accuracy
            worst_grid = g
    print ("Final Result")
    print ("best accuracy", best_accuracy, ", Grid:", best_grid)
    print ("worst accuracy", worst_accuracy, ", Grid:", worst_grid)
    return best_grid

### Parameters being tuned:
1. n_neighbors        = number of nearset neighbouring observation to consider for deciding on the prediciton
2. weights       = should all neighbours have equal weight or weighted by the distance to the observation


In [ ]:
%%time
# Note: previous runs showed that uniform weighting is consistently better than distanced based weighting
# hence due to slow perfromance of this classifier, we avoid the distance based weighting option
if CALC_PARAMS:
    if dev_mode:
        knn_params_grid ={'n_neighbors': [500], 
                         'weights': ['uniform']
                        }
    else:
        knn_params_grid ={'n_neighbors': [500, 1000, 1500], 
                 'weights': ['uniform']
                }
    best_knn_params = find_best_params_knn(knn_params_grid, X_train, y_train, X_val, y_val)
    best_knn_params
else:
    best_knn_params = get_config('knn')

In [ ]:
knn_pipeline = make_knn_pipe_line(params = best_knn_params)
knn_pipeline.fit(X_train, y_train)

In [ ]:
knn_results = evaulate_pipeline(knn_pipeline, X_val, y_val, best_knn_params);
knn_df = pd.DataFrame(knn_results, index= ['knn'])
knn_df

# summary of model metrics

In [ ]:

metrics_df = pd.concat([rf_df, knn_df,catboost_df])
metrics_df.sort_values(by='AUC')

if CALC_PARAMS:
    now = datetime.datetime.now()
    now_string = now.strftime("%m_%d_%Y__%H_%M")
    metrics_df.reset_index().to_csv(f"configuration_{now_string}_sample_{SAMPLE}.csv", index=False)
metrics_df

# Exaplinable AI:  Shap Values

In [ ]:
model = catboost_pipe


# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X_train)
shap.initjs()


Shap Values (Shapely Additive exPlanations) is a model Agnostic method for explaining model predictions
It is based on theory developed by Loid Shapley in the 1950's 


## Shap Summary Plot

In [ ]:

shap.summary_plot(shap_values)

#### explanation
This graph is actually a series of dot-plots:every observation(row) is representated as a dot
for each variable in the model we have a dot-plot showing a dot for each observation in the train set
X-axis: The shap value i.e. the negative or positive contribution of the observation's variable to the final prediction
Y-axis: in case where many observation have the same shap value the dots are stacked vertically
Color: For continuous variables the color represent whether the value of the variable is high or low in the observation

##### Example: TODO verify
in the device_diag variable we see that large screens (strong red) are associated with increased (positive) probability for a click

In [ ]:

def explain_simple(idx, X_train):
    data = X_train.iloc[[idx]]
    shap_values = explainer.shap_values( data)
    return (shap.force_plot(explainer.expected_value,shap_values , data, link='logit'))

## 3 random predictions local explanations

In [ ]:
explain_simple(random.randint(1, X_train.shape[0]), X_train)

In [ ]:
explain_simple(random.randint(1, X_train.shape[0]), X_train)

In [ ]:
explain_simple(random.randint(1, X_train.shape[0]), X_train)

# Section 8: Prediction on external data set

In [ ]:
model = catboost_pipe


In [ ]:
testset = pd.read_csv("ctr_dataset_test.csv")

# Apply preprocessing pipeline

In [ ]:
%%time

X_test, _ = preprocessing_pipeline(testset, one_hot_encoding=USE_SMOTE, allowed_levels_dict = allowed_levels_dict)




### add empty dummy columns for levels that are missing in the predicitons data set

In [ ]:
MODEL_COLS.remove('clicked')
unknown_cols = [c for c in MODEL_COLS  if c not in X_test.columns ]
for c in unknown_cols:
    print(f"adding default column for {c}")
    X_test.loc[:, c] = 0

In [ ]:
print ([c for c in X_test.columns if c not in MODEL_COLS])
X_test = X_test[MODEL_COLS]


In [ ]:
y_hat = model.predict_proba(X_test)

In [ ]:
best_threshold_catboost = catboost_df['threshold'].values[0]
best_threshold_catboost

In [ ]:
y_pred = (y_hat[:,1] >= best_threshold_catboost).astype(int)
predictions = pd.Series(y_pred)

#### Total predictions , total clicks predicted,  Percentage of predicted clicks:

In [ ]:
f"N={predictions.count()}, Clicks={predictions.sum()}, Clicks Percent= {round(predictions.mean()*100,3)}%"

# Group 8 output

In [ ]:
predictions.to_csv('output_8.txt', index=False)

In [ ]:
datetime.datetime.now()